<a href="https://colab.research.google.com/github/thecodeeagle/Genetic-Algorithm-Based-Ensemble-for-Driver-Distraction-Recognition/blob/main/GAEnsemble2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os 
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel

In [ ]:
def weighted_ensemble(weights, models, inputs):
    # Assigning empty array to store 2D array of model predictions
    predictions = []
    predictionsSum = 0
  
    # Loop through all models

    for i in range(len(models)):
       labels = models[i].predict(inputs)

       predictions.append(labels)
       predictionsSum += (labels*weights[i])
      
      

    # Sum of weighted predictions
    predictions[i] = [x * weights[i] for x in predictions[i]]
                      
    return predictionsSum

In [ ]:
import numpy as np

def cal_pop_fitness(fitness_func, pop):
    
    fitness = fitness_func(pop)
    return fitness


def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next
    # #generation.
    
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.min(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
        print(parents.shape)
    return parents


def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k % parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1) % parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.uniform(0, 1, 1)
        offspring_crossover[idx-1, offspring_crossover.shape[0]-1] = \
            offspring_crossover[idx-1, offspring_crossover.shape[0]-1] + random_value
    return offspring_crossover

In [ ]:
def ensemble_fitness(weights, models, inputs, targets, value):
    import numpy as np
    import sklearn
    from sklearn import linear_model
    from sklearn import metrics
    

    fitness = []

    # Normalise weights
    weights = sklearn.preprocessing.normalize(weights, axis=1, norm='l1')

    for i in range(len(weights)):
        predictionsSum = weighted_ensemble(weights[i-1], models, inputs)

        # Calculating bias and variance for use in error if selected
        bias = (np.mean(predictionsSum)-np.mean(targets))**2
        variance = np.var(predictionsSum-targets)

        # Setting output fitness value
        if value == "mse":
            ensembleFit = metrics.mean_squared_error(predictionsSum, targets)
        elif value == "mae":
            ensembleFit = metrics.mean_absolute_error(predictionsSum, targets)
        elif value == "bias":
            ensembleFit = bias
        elif value == "variance":
            ensembleFit = variance
        elif value == "error":
            ensembleFit = bias+variance
        elif value == "log_loss":
            ensembleFit = metrics.log_loss()
        else:
            # If error with input then set it to mse as default
            ensembleFit = metrics.mean_squared_error(predictionsSum, targets)

        fitness.append(ensembleFit)

    # Returning fitness value to minimise
    return fitness

In [5]:
%cd drive/My Drive

/content/drive/My Drive


In [ ]:
from tensorflow import keras
model1 = keras.models.load_model('new_ensemble_effnet_2')
model2 = keras.models.load_model('new_vanillaCNN')
model3 = keras.models.load_model('new_alexnet')
model4 = keras.models.load_model('newd_inceptionv3')
model5 = keras.models.load_model('new_vgg')
model6 = keras.models.load_model('new_densenet3')

In [ ]:
model2.summary()

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as pyplot
from sklearn import linear_model


In [ ]:
loaded = np.load("newdriver.npz")
test_images = loaded["test_images"]
test_labels = loaded["test_labels"]

In [ ]:
import tensorflow as tf
test = tf.keras.utils.to_categorical(test_labels, num_classes=10)

In [ ]:
test_labels.shape

In [ ]:
models = []
models.append(model1)
models.append(model2)
models.append(model3)
models.append(model4)
models.append(model5)
models.append(model6)


In [ ]:
# Create objective function
objective_function = lambda w: ensemble_fitness(w, models, test_images, test, 'mse')

In [ ]:
# Set Genetic Algorithm parameters
sol_per_pop = 6
num_parents_mating = 3
# Defining population size
pop_size = (sol_per_pop, len(models))
print(pop_size)
# Creating the initial population

new_population = np.random.uniform(low=0, high=1, size=pop_size)
print(new_population)
#new_population = np.load("drive/My Drive/np2.npy")

(6, 3)
[[0.75541172 0.98754322 0.49410283]
 [0.09499213 0.24923789 0.74603505]
 [0.65679673 0.38242468 0.83737382]
 [0.08425166 0.09971454 0.77465574]
 [0.77971041 0.19239526 0.09672093]
 [0.19331701 0.34398595 0.25154385]]


In [ ]:
new_population

In [ ]:
for generation in range(50):
    print("Generation: ", generation)
    # Measuring the fitness of each chromosome in the population
    fitness = cal_pop_fitness(objective_function, new_population)

    # Selecting the best parents in the population for mating
    parents = select_mating_pool(new_population, fitness, num_parents_mating)

    # Generating next generation using crossover
    offspring_crossover = crossover(parents, offspring_size=(parents.shape[0], len(models)))

    # Adding some variations to the offspring using mutation
    offspring_mutation = mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration
    #print("NOTHING")
    print("FITNESS VALUE:", fitness)

In [ ]:
new_population = new_population['arr_0']

In [ ]:
# Get the best solution after all generations
fitness = cal_pop_fitness(objective_function, new_population)
# Return the index of that solution and corresponding best fitness
best_match_idx = np.where(fitness == np.min(fitness))
best_match = list(best_match_idx[0])
print(best_match)
print(fitness)

In [ ]:
# Return weights
import matplotlib.pyplot as pyplot

weights = new_population[int(best_match[0])]
# Display optimised network ensemble accuracy details
results = weighted_ensemble(weights, models, test_images)
pyplot.scatter(results, test_labels)
pyplot.show()

In [ ]:
new_population

In [ ]:
weights = new_population[int(best_match[0])]

In [ ]:
weights

In [ ]:
from keras.utils import to_categorical
test_labels = to_categorical(test_labels,10)

In [ ]:
i=0
sum = 0.00
while(i<6):
  sum+= (models[i].predict(test_images))*weights[i]
  i+=1


In [ ]:
final = np.argmax(sum/np.sum(weights), axis = 1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(final, test_labels)

In [ ]:
tolerance = 1e-10
accuracy = (np.abs(y_pred - test) < tolerance ).all(axis=(0,2)).mean()

In [ ]:
np.savez("drive/My Drive/ensemble",new_population)

In [ ]:
loaded = np.load['drive/My Drive/ensemble']

In [ ]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(40, 40)) 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
%matplotlib inline

import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cm_plot_labels = ['Safe Driving','Text Right','Phone Right', 'Text Left', 'Phone_Left','Adjust Radio', 'Drinking', 'Reaching Behind', 'Hair or Makeup', 'Talking to Passenger']
cm= confusion_matrix(y_true = test_labels, y_pred = final)

plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix', normalize = True)